In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

C:\Users\Ghazy\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
data = pd.read_csv(r'E:\Self study\Data\candyhierarchy2017.csv' , encoding='latin-1') 
pd.set_option('display.max_rows',None)
data.head()

,Internal ID,Q1: GOING OUT?,Q2: GENDER,Q3: AGE,Q4: COUNTRY,"Q5: STATE, PROVINCE, COUNTY, ETC",Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,...,Q8: DESPAIR OTHER,Q9: OTHER COMMENTS,Q10: DRESS,Unnamed: 113,Q11: DAY,Q12: MEDIA [Daily Dish],Q12: MEDIA [Science],Q12: MEDIA [ESPN],Q12: MEDIA [Yahoo],"Click Coordinates (x, y)"
0,90258773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,90272821,No,Male,44,USA,NM,MEH,DESPAIR,JOY,MEH,...,NaN,Bottom line is Twix is really the only candy w...,White and gold,NaN,Sunday,NaN,1.0,NaN,NaN,"(84, 25)"
2,90272829,NaN,Male,49,USA,Virginia,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,90272840,No,Male,40,us,or,MEH,DESPAIR,JOY,MEH,...,NaN,Raisins can go to hell,White and gold,NaN,Sunday,NaN,1.0,NaN,NaN,"(75, 23)"
4,90272841,No,Male,23,usa,exton pa,JOY,DESPAIR,JOY,DESPAIR,...,NaN,NaN,White and gold,NaN,Friday,NaN,1.0,NaN,NaN,"(70, 10)"


In [3]:
data.info(verbose= True  , null_counts =  True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2460 entries, 0 to 2459
Data columns (total 120 columns):
 #   Column                                                                                 Non-Null Count  Dtype  
---  ------                                                                                 --------------  -----  
 0   Internal ID                                                                            2460 non-null   int64  
 1   Q1: GOING OUT?                                                                         2350 non-null   object 
 2   Q2: GENDER                                                                             2419 non-null   object 
 3   Q3: AGE                                                                                2376 non-null   object 
 4   Q4: COUNTRY                                                                            2396 non-null   object 
 5   Q5: STATE, PROVINCE, COUNTY, ETC                                           

In [4]:
data.isna().sum().tail(10)

Q8: DESPAIR OTHER           1743
Q9: OTHER COMMENTS          2074
Q10: DRESS                   744
Unnamed: 113                2451
Q11: DAY                     725
Q12: MEDIA [Daily Dish]     2375
Q12: MEDIA [Science]        1098
Q12: MEDIA [ESPN]           2361
Q12: MEDIA [Yahoo]          2393
Click Coordinates (x, y)     855
dtype: int64

In [5]:
col_drops = {"Internal ID" , "Click Coordinates (x, y)","Q12: MEDIA [ESPN]","Q12: MEDIA [Daily Dish]","Unnamed: 113","Q9: OTHER COMMENTS" , "Q8: DESPAIR OTHER","Q7: JOY OTHER" , "Q12: MEDIA [Yahoo]"}
for x in col_drops :
    data.drop(x , axis = 1 , inplace = True)

# GOING OUT

In [6]:
data["Q1: GOING OUT?"].value_counts()

No     2038
Yes     312
Name: Q1: GOING OUT?, dtype: int64

In [7]:
data["Q1: GOING OUT?"].isna().sum()

110

In [8]:
going_out_mode = data["Q1: GOING OUT?"].mode()[0]
data["Q1: GOING OUT?"].fillna(going_out_mode , inplace = True)
data["Q1: GOING OUT?"].value_counts()

No     2148
Yes     312
Name: Q1: GOING OUT?, dtype: int64

# GENDER 

In [9]:
data["Q2: GENDER"].value_counts()

Male                  1467
Female                 839
I'd rather not say      83
Other                   30
Name: Q2: GENDER, dtype: int64

In [10]:
data["Q2: GENDER"].isna().sum()

41

In [11]:
data["Q2: GENDER"].fillna("I'd rather not say" , inplace = True)
data["Q2: GENDER"].value_counts()

Male                  1467
Female                 839
I'd rather not say     124
Other                   30
Name: Q2: GENDER, dtype: int64

# Age

In [12]:
data["Q3: AGE"].value_counts()

40                                                  92
34                                                  90
37                                                  89
43                                                  86
42                                                  79
36                                                  79
35                                                  77
44                                                  76
38                                                  76
33                                                  75
45                                                  75
41                                                  74
50                                                  71
39                                                  69
49                                                  69
32                                                  68
48                                                  64
47                                                  63
30        

In [13]:
data["Q3: AGE"].unique()

array([nan, '44', '49', '40', '23', '53', '33', '43', '56', '64', '37',
       '59', '48', '54', '36', '45', '25', '34', '35', '38', '58', '50',
       '47', '16', '52', '63', '65', '41', '27', '31', '61', '46', '42',
       '62', '29', '39', '32', '28', '69', '67', '30', '22', '26', '51',
       '70', '24', '18', '19', 'Old enough', '57', '60', '66', '12',
       'Many', '55', '72', '?', '21', '11', 'no', '9', '68', '20', '6',
       '10', '71', '90', '13', '45-55', '312', '99', '7', 'hahahahaha',
       '88', '39.4', '74', '102', 'older than dirt', '17', '15', '8',
       '75', '5u', 'Enough', 'See question 2', '24-50', '14', 'Over 50',
       '100', '76', '1000', 'sixty-nine', '46 Halloweens.', 'ancient',
       '77', 'OLD', 'old', '73', '70 1/2', '1', 'MY NAME JEFF', '4',
       '59 on the day after Halloween', 'old enough', 'your mom',
       'I can remember when Java was a cool new language', '60+'],
      dtype=object)

In [14]:
age_drops = {'Old enough','45-55','24-50','?','no','Many','hahahahaha','older than dirt','Enough','See question 2','old','ancient','old enough','5u','46 Halloweens.','sixty-nine','Over 50','OLD','MY NAME JEFF','59 on the day after Halloween','your mom','I can remember when Java was a cool new language', '60+', '1000', '70 1/2', '1', '4','312'}
for x in age_drops:
    data['Q3: AGE'].replace(x , np.nan,inplace=True)

In [15]:
age_median = data['Q3: AGE'].median()
data['Q3: AGE'].fillna(age_median , inplace = True)

In [16]:
data["Q3: AGE"].unique()

array([41.0, '44', '49', '40', '23', '53', '33', '43', '56', '64', '37',
       '59', '48', '54', '36', '45', '25', '34', '35', '38', '58', '50',
       '47', '16', '52', '63', '65', '41', '27', '31', '61', '46', '42',
       '62', '29', '39', '32', '28', '69', '67', '30', '22', '26', '51',
       '70', '24', '18', '19', '57', '60', '66', '12', '55', '72', '21',
       '11', '9', '68', '20', '6', '10', '71', '90', '13', '99', '7',
       '88', '39.4', '74', '102', '17', '15', '8', '75', '14', '100',
       '76', '77', '73'], dtype=object)

In [17]:
data["Q3: AGE"] = data["Q3: AGE"].astype('float')

# Country

In [18]:
data["Q4: COUNTRY"].unique()

array([nan, 'USA ', 'USA', 'us', 'usa', 'canada', 'Canada', 'Us', 'US',
       'Murica', 'United States', 'uk', 'United Kingdom', 'united states',
       'Usa', 'United States ', 'United staes',
       'United States of America', 'UAE', 'England', 'UK', 'canada ',
       'Mexico', 'United states', 'u.s.a.', 'USAUSAUSA', 'america', '35',
       'france', 'United States of America ', 'U.S.A.', 'finland',
       'unhinged states', 'Canada ', 'united states of america',
       'US of A', 'Unites States', 'The United States', 'North Carolina ',
       'Unied States', 'Netherlands', 'germany', 'Europe', 'Earth', 'U S',
       'u.s.', 'U.K. ', 'Costa Rica', 'The United States of America',
       'unite states', 'U.S.', '46', 'cascadia', 'Australia',
       'insanity lately', 'Greece', 'USA? Hard to tell anymore..',
       "'merica", 'usas', 'Pittsburgh', '45', 'United State', '32',
       'France', 'australia', 'A', 'Can', 'Canae', 'New York',
       'Trumpistan', 'Ireland', 'United Sates', '

In [19]:
data["Q4: COUNTRY"] = data["Q4: COUNTRY"].str.strip()
data["Q4: COUNTRY"] = data["Q4: COUNTRY"].str.upper()

In [20]:
data["Q4: COUNTRY"].value_counts()

USA                                                                     1130
UNITED STATES                                                            592
CANADA                                                                   224
US                                                                       182
UNITED STATES OF AMERICA                                                  73
UK                                                                        18
UNITED KINGDOM                                                            14
U.S.                                                                      12
GERMANY                                                                   10
U.S.A.                                                                     8
AUSTRALIA                                                                  7
AMERICA                                                                    6
NETHERLANDS                                                                6

In [21]:
x = []
countries = ['USA','CANADA','UK','UAE','ENGLAND','MEXICO','FRANCE','FINLAND','NORTH CAROLINA','NETHERLANDS','GERMANY','TAIWAN' , 'UNITED STATES']
for country in countries:
    matches = process.extract(country, data['Q4: COUNTRY'], limit = data.shape[0])
    for potential_match in matches:
        if (potential_match[1] >= 70):
            data.loc[data['Q4: COUNTRY'] == potential_match[0],'Q4: COUNTRY'] = country

In [22]:
USA_Names = ['UNITED STATES' ,'US OF A' , 'N. AMERICA' ,'MURICA' ,'MURRIKA' ,'USA? HARD TO TELL ANYMORE..' ,'NEW JERSEY' , 'AHEM....AMERCA' , "'MERICA" , 'CALIFORNIA' ,'TRUMPISTAN']
Non_Country_Names = ['INSANITY LATELY' , '32' , 'EARTH' , '46' , 'FEAR AND LOATHING' ,'NARNIA' , 'SUBSCRIBE TO DM4UZ3 ON YOUTUBE' ,'45' , '1' ,'35','EUROPE' , "I DON'T KNOW ANYMORE" , 'SOVIET CANUCKISTAN']

for x in USA_Names:
    data['Q4: COUNTRY'].replace(x , 'USA',inplace=True)


for y in Non_Country_Names:
    data['Q4: COUNTRY'].replace(y , np.nan,inplace=True)
    
data['Q4: COUNTRY'].replace('UNITED KINGDOM' , 'UK',inplace=True)

In [23]:
data["Q4: COUNTRY"].unique()

array([nan, 'USA', 'CANADA', 'UK', 'UAE', 'FINLAND', 'MEXICO', 'AMERICA',
       'FRANCE', 'NORTH CAROLINA', 'NETHERLANDS', 'GERMANY', 'COSTA RICA',
       'AUSTRALIA', 'GREECE', 'PITTSBURGH', 'NEW YORK', 'KOREA', 'JAPAN',
       'SOUTH AFRICA', 'SCOTLAND', 'DENMARK', 'SWITZERLAND', 'UD',
       'SOUTH KOREA', 'INDONESIA', 'ATLANTIS', 'ALASKA', 'SINGAPORE',
       'CHINA', 'TAIWAN', 'HONG KONG', 'SPAIN', 'SWEDEN'], dtype=object)

# State

In [24]:
data['Q5: STATE, PROVINCE, COUNTY, ETC'] = data['Q5: STATE, PROVINCE, COUNTY, ETC'].str.strip()
data['Q5: STATE, PROVINCE, COUNTY, ETC'] = data['Q5: STATE, PROVINCE, COUNTY, ETC'].str.upper()

In [25]:
len(data['Q5: STATE, PROVINCE, COUNTY, ETC'].unique())

332

# Multiple choices questions

In [26]:
for Col_Name in data.loc[:,'Q6 | 100 Grand Bar':'Q12: MEDIA [Science]'] :
    data[Col_Name] = data[Col_Name].astype("category")
    data[Col_Name].fillna(data[Col_Name].mode()[0], inplace=True)